In [217]:
import pandas as pd
import numpy as np
import nltk

from sklearn.feature_extraction.text import CountVectorizer


In [218]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sonia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [219]:
df = pd.read_csv('bbc_text_cls.csv')
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [220]:
vectorizer = CountVectorizer()

# Fit and transform your documents into term-frequency matrix
tf = vectorizer.fit_transform(df['text'])

# The dictionary mapping words to index
word2index = vectorizer.vocabulary_


In [221]:
# Reverse mapping
index2word = {v:k for k, v in word2index.items()}

In [222]:
# Number of docs
N = len(df['text'])

In [223]:
# Number of words
V = len(word2index)

In [224]:
np.random.seed(8008)

In [225]:
document_freq = np.sum((tf.toarray() > 0).astype(int), axis=0)
idf = np.log(N / (1 + document_freq))

# Now for the multiplication of tf and idf. As tf is sparse use the multiply function:
tf_idf = tf.multiply(idf)
tf_idf_csr = tf_idf.tocsr() # Convert to csr_matrix before subscripting

# Check a random doc, show the top 5 terms in terms of TF-IDF score
i = np.random.choice(N)
row = df.iloc[i]
print("Label:", row['labels'])
print("Text:", row['text'].split("\n", 1)[0])
print("Top 5 terms:")
scores = tf_idf_csr[i].toarray().ravel() # Extracting the row vector and flattening to a 1D Numpy array
indices = np.argsort(-scores)
for j in indices[:5]:
    word = index2word[int(j)] # Convert numpy int to regular int
    print(word)

Label: sport
Text: IAAF awaits Greek pair's response
Top 5 terms:
iaaf
thanou
kenteris
greek
tests
